## Building Tilesets using Datashader

Datashader provides `render_tiles` which is a utility function for creating tilesets from arbitrary datashader pipelines.

In [ ]:
from datashader.tiles import render_tiles

A couple of notes about the tiling process:
    
- By default, uses a simple `Web Mercator Tiling Scheme (EPSG:3857)`
- call `render_tiles` with the following arguments:

```python
extent_of_area_i_want_to_tile = (-500000, -500000, 500000, 500000)  # xmin, ymin, xmax, ymax
render_tiles(extent_of_data_i_want_to_handle,
             tile_levels=range(6),
             output_path='example_tileset_output_directory',
             load_data_func=function_which_returns_dataframe,
             ds_pipeline_func=function_which_renders_dataframe_to_datashader_image,
             post_render_func=function_which_post_processes_image)
```

- data representing x / y coordinates is assumed to be represented in meters (m) based on the Web Mercator coordinate system.
- the tiling extent is subdivided into `supertiles` generally of size `4096 x 4096`
- the `load_data_func` returns a dataframe-like object and contains your data access specific code.
- the `ds_pipeline_func` returns a (agg, img) tuple and contains your datashader specific code.
- the `post_render_func` is called once for each final tile (`default 256 x 256`) and contains PIL (Python Imaging Library) specific code.  This is the hook for adding additional filters, text, watermarks, etc. to output tiles.


  

## Creating Tile Component Functions

### Create `load_data_func`
- accepts `x_range` and `y_range` arguments which correspond to the ranges of the supertile being rendered.
- returns a dataframe-like object (pd.Dataframe / dask.Dataframe)

In [ ]:
import pandas as pd
import numpy as np

df = None
def load_data_func(x_range, y_range):
    global df
    if df is None:
        xs = np.random.normal(loc=0, scale=500000, size=10000000)
        ys = np.random.normal(loc=0, scale=500000, size=10000000)
        df = pd.DataFrame(dict(x=xs, y=ys))

    return df.loc[df['x'].between(*x_range) & df['y'].between(*y_range)]

### Create `ds_pipeline_func`

- accepts `df`, `x_range`, `y_range`, `plot_height`, `plot_width`, `span` arguments which correspond to the data, ranges, and plot dimensions of the supertile being rendered.  The `span` argument can be forwarded on to `transfer_functions.shade` to disable color auto-ranging and set a fixed min/max for interpolating color values.
- returns a tuple of aggregate `(xr.DataArray)` and image `(datashader.Image)`
- The return aggregate can be used for stats sampling

In [ ]:
import datashader as ds
import datashader.transfer_functions as tf
from datashader.colors import viridis

def ds_pipeline_func(df, x_range, y_range, plot_height, plot_width, span=None):
    
    # aggregate
    cvs = ds.Canvas(x_range=x_range, y_range=y_range,
                    plot_height=plot_height, plot_width=plot_width)
    agg = cvs.points(df, 'x', 'y')
    
    # shade
    img = tf.shade(agg, cmap=viridis, span=span)
    img = tf.set_background(img, 'black')
    return agg, img

### Create `post_render_func`
- accepts `img `, `extras` arguments which correspond to the output PIL.Image before it is write to disk (or S3), and addtional image properties.
- returns image `(PIL.Image)`
- this is a good place to run any non-datashader-specific logic on each output tile.

In [ ]:
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageFilter

def post_render_func(img, extras):
    info = "x={} / y={} / z={} / w={} / h={}".format(extras['x'], extras['y'], extras['z'],
                                                     img.width, img.height)

    draw = ImageDraw.Draw(img)
    draw.text((5, 5), info, fill='rgb(255, 255, 255)')
    return img.filter(ImageFilter.CONTOUR)

### Call `render_tiles`

In [ ]:
full_extent_of_data = (-500000, -500000, 500000, 500000)
output_path = 'test_tiles_output'
results = render_tiles(full_extent_of_data,
                        range(12),
                        load_data_func=load_data_func,
                        ds_pipeline_func=ds_pipeline_func,
                        post_render_func=post_render_func,
                        output_path=output_path)

### Preview the tileset using Bokeh

In [ ]:
from bokeh.plotting import figure
from bokeh.models.tiles import WMTSTileSource
from bokeh.io import show
from bokeh.io import output_notebook
from bokeh.models import Button
from bokeh.models import CustomJS, Button
from bokeh.models import BoxZoomTool
from bokeh.layouts import widgetbox, column

output_notebook()

width = 800
height = 800

xmin, ymin, xmax, ymax = full_extent_of_data

p = figure(width=width, height=height, 
           x_range=(int(-20e6), int(20e6)),
           y_range=(int(-20e6), int(20e6)),
           tools="pan,wheel_zoom,reset")

p.axis.visible = False
p.add_tools(BoxZoomTool(match_aspect=True))
p.add_tile(WMTSTileSource(url="http://localhost:8080/{Z}/{X}/{Y}.png"))
show(p)